# WORD VECTORS FEATURES - XGBOOST

In [1]:
import numpy as np
import pandas as pd
import os
import re
import tqdm

In [4]:
#os.chdir(r'C:\Users\Helio\Documents\akira\ECONO3_TUTORIAIS\Scrapings\\')
os.chdir(r'C:\Users\C336682\Documents\fake\IC_FakeNews-master\data\new\\')

In [5]:
data = pd.read_csv('news_concat_01.csv').drop(columns=['Unnamed: 0'])

In [312]:
str(data.iloc[22200,1])

'Presidente da Croácia de biquíni na praia #'

In [6]:
data.head()

,Label,Title
0,1,Indústria brasileira reage com melhora do comé...
1,1,A bancarrota de Detroit deixa no ar as pensões...
2,1,"PIB no Brasil cai 0,5% na leitura trimestral, ..."
3,1,O órgão supervisor europeu questiona o trabalh...
4,1,"Vega Sicilia, a ilusão da escassez"


In [7]:
from gensim.models import KeyedVectors
from nltk import word_tokenize
import nltk
nltk.download('punkt')

C:\Users\C336682\AppData\Local\Continuum\anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\C336682\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [12]:
#os.chdir(r'C:\Users\Helio\Documents\akira\ECONO3_TUTORIAIS\embedding\\')
os.chdir(r'C:\Users\C336682\Documents\fake\IC_FakeNews-master\embeddings\w2v_cbow_s300\\')

In [148]:
#model = KeyedVectors.load_word2vec_format('word2v_cbow_s300.txt', unicode_errors="ignore")
model_w2v = KeyedVectors.load_word2vec_format('cbow_s300.txt', unicode_errors="ignore")

In [14]:
def sent2vec(s, model):
    words = str(s).lower()
    words = word_tokenize(words)
    #words = [w for w in words if not w in stop_words]
    words = [w for w in words if w.isalpha()]
    M = []
    for w in words:
        try:
            M.append(model[w])
        except:
            continue
    M = np.array(M)
    v = M.sum(axis=0)
    if type(v) != np.ndarray:
        return np.zeros(300)
    return v / np.sqrt((v ** 2).sum())

In [24]:
X_w2v_cbow_list = [sent2vec(x, model) for x in data['Title']]

In [25]:
X_w2v_cbow = pd.DataFrame()

for j in range(len(X_w2v_cbow_list)):
    X_w2v_cbow[j] = X_w2v_cbow_list[j]

In [26]:
X_w2v_cbow = X_w2v_cbow.T

In [27]:
columns = []
for j in range(1,301):
    columns.append('feature'+str(j))

In [28]:
X_w2v_cbow.columns = columns

In [29]:
X_w2v_cbow.head()

,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,feature10,...,feature291,feature292,feature293,feature294,feature295,feature296,feature297,feature298,feature299,feature300
0,0.042570,0.012104,0.074938,-0.013384,0.092475,-0.031281,-0.070357,-0.024914,-0.004816,0.013637,...,0.005758,-0.077279,-0.017853,-0.006452,-0.017334,-0.106524,-0.065605,0.046967,-0.037649,-0.061136
1,-0.018268,0.116134,0.088298,-0.064570,0.011260,0.039332,0.083535,0.042699,-0.000626,0.071059,...,-0.057831,-0.023792,0.017571,-0.036500,-0.066888,-0.001920,0.123694,-0.045279,0.013808,-0.082070
2,0.095050,0.034736,-0.040186,-0.048977,0.070348,0.006896,0.065039,0.089740,-0.068805,-0.027196,...,0.003375,0.035696,0.043836,-0.121432,-0.002649,0.018771,0.087685,0.047486,-0.124016,-0.026470
3,0.019449,0.099607,-0.023152,0.035851,-0.018510,0.036798,0.032469,0.067076,0.024612,0.009722,...,0.063799,-0.092861,0.096599,-0.058167,-0.003396,0.069895,0.141475,-0.025738,0.007802,-0.052845
4,0.064135,-0.029280,-0.068144,-0.130111,0.004120,-0.007988,0.058953,0.000371,0.019016,0.033978,...,-0.002356,0.042550,0.035945,0.034265,-0.035605,0.000997,-0.022376,-0.005851,-0.010347,0.033598


In [22]:
import xgboost as xgb
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [32]:
label_encoder = preprocessing.LabelEncoder()
y = label_encoder.fit_transform(data.Label.values)
X = X_w2v_cbow.values

Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, y,
                                            random_state = 42,
                                            test_size = 0.2,
                                            stratify = y) 

In [33]:
clf = xgb.XGBClassifier(nthread=10, silent=False)
clf.fit(Xtrain, Ytrain)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=10, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=False, subsample=1)

In [34]:
clf.score(Xtest, Ytest)

C:\Users\Helio\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.9262800084979818

In [35]:
news = ["Em entrevista, Moro fala em reação, cita risco de morte e dá duríssimo recado aos bandidos so Brasil",
        "Bolsonaro ganha em disparada na contagem de votos do IBGE",
        "Lula condenado à 10 anos de cadeia pelo caso do Triplex",
        "Lula ordena a prisão de Sérgio Moro",
        "Peixe alienígena encontrado no Rio de Janeiro",
        "Lula preso amanhã",
        "Terremoto no Brasil destroi cidades no Alagoas",
        "Dilma reassumiu a presidência após prisão de Michel Temer",
        "Michel Temer assume presidência do Brasil",
        "Toffoli propõe estender restrição ao foro para todas as autoridades",
        "Intenção de votos em Bolsonaro dispara no mês de Outubro"]

In [15]:
def classify(noticia, model):
    
    test=sent2vec(noticia, model)
    vector = np.zeros((1,300))
    
    j=0
    for n in test:
        vector[0,j] = n
        j+=1
    

    prob = round((clf.predict_proba(vector)[0][0]),4)*100
    print("A notícia tem " + str(prob)[:5] + "% de probalidade de ser falsa.")


In [42]:
for n in news:
    print(n)
    classify(n, model)
    print('\n')

Em entrevista, Moro fala em reação, cita risco de morte e dá duríssimo recado aos bandidos so Brasil
A notícia tem 79.96% de probalidade de ser falsa.


Bolsonaro ganha em disparada na contagem de votos do IBGE
A notícia tem 29.08% de probalidade de ser falsa.


Lula condenado à 10 anos de cadeia pelo caso do Triplex
A notícia tem 21.27% de probalidade de ser falsa.


Lula ordena a prisão de Sérgio Moro
A notícia tem 41.31% de probalidade de ser falsa.


Peixe alienígena encontrado no Rio de Janeiro
A notícia tem 42.03% de probalidade de ser falsa.


Lula preso amanhã
A notícia tem 11.71% de probalidade de ser falsa.


Terremoto no Brasil destroi cidades no Alagoas
A notícia tem 18.16% de probalidade de ser falsa.


Dilma reassumiu a presidência após prisão de Michel Temer
A notícia tem 17.50% de probalidade de ser falsa.


Michel Temer assume presidência do Brasil
A notícia tem 14.73% de probalidade de ser falsa.


Toffoli propõe estender restrição ao foro para todas as autoridades
A 

## Iterate through different word vectors: Word2Vec, Glove, Wang2Vec

In [59]:
vectors =['word2v_cbow_s300.txt',
          'word2v_skip_s300.txt',
          'fasttext_cbow_s300.txt',
          'fasttext_skip_s300.txt',
          'wang2v_cbow_s300.txt',
          'wang2v_skip_s300.txt',
          'glove_s300.txt']

In [50]:
X_word_vectors = []
Keyedvectors = []

In [58]:
for vector in vectors:
    
    model = KeyedVectors.load_word2vec_format(vector, unicode_errors="ignore")
    Keyedvectors.append(model)
    
    X_word_vector_list = [sent2vec(x, model) for x in data['Title']]
    
    X_word_vector = pd.DataFrame()

    for j in range(len(X_word_vector_list)):
        X_word_vector[j] = X_word_vector_list[j]
    
    X_word_vector = X_word_vector.T
    
    columns = []
    for j in range(1,301):
        columns.append('feature'+str(j))
    
    X_word_vector.columns = columns
    
    X_word_vectors.append(X_word_vector)

In [21]:
import warnings
warnings.filterwarnings('ignore')

In [61]:
j=0

for el in X_word_vectors:
    
    label_encoder = preprocessing.LabelEncoder()
    y = label_encoder.fit_transform(data.Label.values)
    X = el.values

    Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, y,
                                                random_state = 42,
                                                test_size = 0.2,
                                                stratify = y) 
    
    clf = xgb.XGBClassifier(nthread=10, silent=False)
    print(vectors[j])
    clf.fit(Xtrain, Ytrain)
    print(clf.score(Xtest, Ytest))
    print('\n')
    
    j+=1
    
    #BEST: WORD2VEC-SKIPGRAM

word2v_cbow_s300.txt
0.9262800084979818


word2v_skip_s300.txt
0.9379647333758232


fasttext_cbow_s300.txt
0.9354153388570214


fasttext_skip_s300.txt
0.9343530911408541


wang2v_cbow_s300.txt
0.9335032929679201


wang2v_skip_s300.txt
0.934777990227321


glove_s300.txt
0.9326534947949862




In [62]:
clf

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=10, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=False, subsample=1)

In [63]:
#TODO: TUNE HYPERPARAMETERS

## NEURAL NETWORKS WITH WORDVEC-SKIPGRAM FEATURES

In [18]:
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping

Using TensorFlow backend.


In [82]:
model = KeyedVectors.load_word2vec_format(vectors[1], unicode_errors="ignore")

In [19]:
x_w2v = [sent2vec(x, model) for x in data['Title']]
x_w2v = np.array(x_w2v)

In [131]:
label_encoder = preprocessing.LabelEncoder()
y = label_encoder.fit_transform(data.Label.values)

Xtrain, Xtest, Ytrain, Ytest = train_test_split(x_w2v, y,
                                                random_state = 42,
                                                test_size = 0.2,
                                                stratify = y) 

In [132]:
scl = preprocessing.StandardScaler()
xtrain_w2v_scl = scl.fit_transform(Xtrain)
xtest_w2v_scl = scl.transform(Xtest)

In [133]:
ytrain_enc = np_utils.to_categorical(Ytrain)
ytest_enc = np_utils.to_categorical(Ytest)

In [236]:
from keras import optimizers

In [276]:
model = Sequential()

model.add(Dense(200, input_dim=300, activation='relu', ))
model.add(Dropout(0.5))
model.add(BatchNormalization())

model.add(Dense(100, activation='relu'))
model.add(Dropout(0.7))
model.add(BatchNormalization())

model.add(Dense(2))
model.add(Activation('softmax'))

# compile the model
adamax = optimizers.Adamax(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0)
model.compile(loss='binary_crossentropy', optimizer=adamax, metrics=['accuracy'])

In [277]:
model.fit(xtrain_w2v_scl, y=ytrain_enc, batch_size=64, 
          epochs=10, verbose=1,
          validation_data=(xtest_w2v_scl, ytest_enc))

Train on 20787 samples, validate on 5197 samples
Epoch 1/10
20787/20787 [==============================] - ETA: 9:21 - loss: 1.0647 - acc: 0.546 - ETA: 37s - loss: 0.9157 - acc: 0.560 - ETA: 19s - loss: 0.8427 - acc: 0.59 - ETA: 13s - loss: 0.7774 - acc: 0.62 - ETA: 9s - loss: 0.7228 - acc: 0.6585 - ETA: 7s - loss: 0.6829 - acc: 0.678 - ETA: 5s - loss: 0.6505 - acc: 0.697 - ETA: 5s - loss: 0.6241 - acc: 0.711 - ETA: 4s - loss: 0.6037 - acc: 0.723 - ETA: 3s - loss: 0.5854 - acc: 0.732 - ETA: 3s - loss: 0.5710 - acc: 0.739 - ETA: 3s - loss: 0.5550 - acc: 0.748 - ETA: 2s - loss: 0.5405 - acc: 0.756 - ETA: 2s - loss: 0.5229 - acc: 0.766 - ETA: 1s - loss: 0.5128 - acc: 0.771 - ETA: 1s - loss: 0.5004 - acc: 0.777 - ETA: 1s - loss: 0.4934 - acc: 0.781 - ETA: 1s - loss: 0.4861 - acc: 0.786 - ETA: 1s - loss: 0.4797 - acc: 0.789 - ETA: 1s - loss: 0.4726 - acc: 0.793 - ETA: 0s - loss: 0.4653 - acc: 0.796 - ETA: 0s - loss: 0.4590 - acc: 0.799 - ETA: 0s - loss: 0.4496 - acc: 0.804 - ETA: 0s - loss:

20787/20787 [==============================] - ETA: 0s - loss: 0.0605 - acc: 0.984 - ETA: 1s - loss: 0.1432 - acc: 0.944 - ETA: 1s - loss: 0.1620 - acc: 0.935 - ETA: 1s - loss: 0.1684 - acc: 0.936 - ETA: 1s - loss: 0.1737 - acc: 0.935 - ETA: 1s - loss: 0.1737 - acc: 0.933 - ETA: 1s - loss: 0.1768 - acc: 0.930 - ETA: 1s - loss: 0.1740 - acc: 0.932 - ETA: 1s - loss: 0.1722 - acc: 0.933 - ETA: 1s - loss: 0.1687 - acc: 0.934 - ETA: 1s - loss: 0.1708 - acc: 0.933 - ETA: 1s - loss: 0.1713 - acc: 0.932 - ETA: 0s - loss: 0.1721 - acc: 0.932 - ETA: 0s - loss: 0.1731 - acc: 0.932 - ETA: 0s - loss: 0.1733 - acc: 0.932 - ETA: 0s - loss: 0.1721 - acc: 0.932 - ETA: 0s - loss: 0.1698 - acc: 0.933 - ETA: 0s - loss: 0.1700 - acc: 0.933 - ETA: 0s - loss: 0.1699 - acc: 0.933 - ETA: 0s - loss: 0.1690 - acc: 0.934 - ETA: 0s - loss: 0.1695 - acc: 0.934 - ETA: 0s - loss: 0.1688 - acc: 0.934 - ETA: 0s - loss: 0.1691 - acc: 0.934 - ETA: 0s - loss: 0.1694 - acc: 0.934 - ETA: 0s - loss: 0.1714 - acc: 0.934 - ETA

### ESTÁ ESTRANHO, CLASSIFICANDO TUDO COMO ZERO

In [325]:
vec = sent2vec("# # # # # #", model_w2v)

In [326]:
vec.shape

(300,)

In [327]:
vector = np.zeros((1,300))
    
j=0
for n in vec:
    vector[0,j] = n
    j+=1

In [329]:
model.predict_classes(vector)

array([1], dtype=int64)

In [330]:
model.predict_proba(vector)

array([[0.14055635, 0.8594436 ]], dtype=float32)

In [331]:
model.evaluate(xtest_w2v_scl, ytest_enc, verbose=0)

[0.13137345122180955, 0.9505483933038291]

In [284]:
news = ["Em entrevista, Moro fala em reação, cita risco de morte e dá duríssimo recado aos bandidos so Brasil",
        "Bolsonaro ganha em disparada na contagem de votos do IBGE",
        "Lula condenado à 10 anos de cadeia pelo caso do Triplex",
        "Lula ordena a prisão de Sérgio Moro",
        "Peixe alienígena encontrado no Rio de Janeiro",
        "Lula preso amanhã",
        "Terremoto no Brasil destroi cidades no Alagoas",
        "Dilma reassumiu a presidência após prisão de Michel Temer",
        "Michel Temer assume presidência do Brasil",
        "Toffoli propõe estender restrição ao foro para todas as autoridades",
        "Intenção de votos em Bolsonaro dispara no mês de Outubro"]

In [285]:
def classify(noticia, model):
    
    test=sent2vec(noticia, model)
    vector = np.zeros((1,300))
    
    j=0
    for n in test:
        vector[0,j] = n
        j+=1
    

    print(model.predict_classes(vector))

In [286]:
for n in news:
    classify(n, model)

[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]


In [ ]:
#tentar NN com TFIDFse